Данная тетрадь посвящена инференсу дообученных моделей.

In [1]:
import os
import gc

import torch
import pandas as pd

from tqdm import tqdm
from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/opt/miniconda3/envs/project_news_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
gold_examples = pd.read_csv('{your_dataset_path}')

gold_examples.drop(columns = ['Unnamed: 0'], inplace = True)

model_instructions = """Ты - профессиональный журналист с многолетним опытом.
Твоя задача - сгенерировать заголовок новостной статьи, который в максимальной степени отражал бы содержание новости.
Не поясняй свой ответ. Твой финальный ответ должен включать в себя только заголовок и ничего более.
"""

gold_examples['Инструкции модели'] = model_instructions

In [3]:
alpaca_prompt = """Ниже представлены следующие аспекты:
1. Пользовательский запрос.
2. Исходные данные - контекст
3. Ответ - твой ответ.
Сгенерируй ответ, который в полной мере выполняет пользовательский запрос.  
### Пользовательский запрос:
{}

### Исходные данные:
{}

### Ответ:
{}"""

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = '{your_finetuned_model_path}', #Укажите директорию, в которую сохранили LoRa-адаптеры
    max_seq_length = 2048,
    dtype=None,
    load_in_4bit=True
)

In [ ]:
FastLanguageModel.for_inference(model)

In [6]:
def model_generate(article_text: str, instructions: str) -> str:
    '''Функция использует инициализированную модель для генерации заголовка новостной статьи
    по тексту новостной статьи.
    
    Args:
        article_text (str): Текст новостной статьи.
        instructions (str): Инструкции модели. Используются для форматирования alpaca_prompt_template
    
    Returns:
        str: сгенерированный новостной заголовок'''
    
    inputs = tokenizer(
        [
            alpaca_prompt.format(
                instructions,
                article_text,
                ""
            )
        ], return_tensors = 'pt').to('cuda')

    outputs = model.generate(**inputs, max_new_tokens = 100, use_cache = True, pad_token_id = tokenizer.eos_token_id)
    result = tokenizer.batch_decode(outputs)
    result = result[0].split('Ответ:', 1)[1].replace('\n', '').replace('</s>', '')

    return result

In [16]:
test_headline = gold_examples['Заголовок'][50]

test_article = gold_examples['Текст новости'][50]

generated_article = model_generate(article_text = test_article,
                                   instructions = model_instructions)

print(f"Оригинальный заголовок: \n```{test_headline}```\n\
      Сгенерированный заголовок: \n```{generated_article}```\n")

Оригинальный заголовок: 
```Налоги больше снижать не будут```
      Сгенерированный заголовок: 
```Власти не снизят налоговую нагрузку на томский бизнес<eos>```



Видим, что даже маленькая модель Gemma2-2b (4bit) генерирует вполне осознанные заголовки. 

Опишем функцию для автоматизации применения моделей для генерации новостных заголовков.

In [4]:
def generate_headline(instructions: str, article_text: str)->str:
    '''Функция выполняет генерацию новостного заголовка заранее инициализированной моделью.
    
    Args:
        instructions (str): Инструкции модели.
        article_text (str): Текст новостной статьи, для которой требуется сгенерировать новостной заголовок.
    
    Returns:
        str - Сгенерированный заголовок.'''
    inputs = tokenizer(
        [
        alpaca_prompt.format(
            instructions,
            article_text,
            ""
            )
        ], return_tensors = 'pt').to('cuda')

    outputs = model.generate(**inputs, max_new_tokens = 100, use_cache = True, pad_token_id = tokenizer.eos_token_id)
    result = tokenizer.batch_decode(outputs)
    result = result[0].split('Ответ:', 1)[1].replace('\n', '').replace('</s>', '').replace('<eos>', '')

    return result

In [5]:
def clear_cache(model, tokenizer) -> None:
    '''Функция удаляет модель из активной памяти GPU'''
    with torch.no_grad():
        model.cpu()
    del model
    del tokenizer
    
    gc.collect()
    
    torch.cuda.empty_cache()

In [ ]:
tqdm.pandas()

models_dirs = ['unsloth/'+x for x in os.listdir('unsloth')]

for model in models_dirs:

    print(f'Выполняется инициализация модели {model}...')

    model_name = model

    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = model,
        max_seq_length = 2048,
        dtype=None,
        load_in_4bit=True
    )

    FastLanguageModel.for_inference(model)

    print(f'Модель {model} инициализирована')

    print(f'Выполняется генерация моделью {model}...')

    gold_examples[f"{model_name}_generated_headline"] = gold_examples['Текст новости'].progress_apply(lambda x: generate_headline(instructions = model_instructions,
                                                                                                                             article_text = x))
    
    print('Выполняется очистка кэша...')

    clear_cache(model = model, tokenizer = tokenizer)

    print('Очистка кеша завершена...')

gold_examples.drop(columns = ['Дата публикации', 'Тэги', 'Инструкции модели'], inplace = True)

gold_examples.to_csv("{your_dataset_path}")

print('Датасет, содержащий генерацию, успешно сохранен')